In [0]:
%sql
select * from spam.spamham;

In [0]:
df = _sqldf

In [0]:
def elim_subject(x):
  return "".join(x.split(":")[1:]).strip()

#t = "Subject: cigarettes wholesale ! hywwzzlzd  $ 19 . 95 and up ! buy cartons of cigarettes wholesale , starting at $ 19 . 95 .  free shipping !  why pay state taxes ? 100 % legal . mailed from swiss bonded warehouse . #for  personal use only , must be 18 years of age and older , verified by credit  card .  aol users click here  to be removed from future mailings , reply to this email with remove in the subject line ."

#print(elim_subject(t))
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

elim_subject_udf = udf(elim_subject, StringType())

df_processed = df.withColumn('message', elim_subject_udf(df['text']))
df_processed.show(100)

In [0]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol='message', outputCol='words')
wordsData = tokenizer.transform(df_processed)
display(wordsData)
wordsData.show(50)

In [0]:
from pyspark.ml.feature import CountVectorizer

df_test = spark.createDataFrame(
[
  (0, "a b c d".split(" ")),
  (1, "a b b c a".split(" "))
], 
["id", "words"])
display(df_test)

In [0]:
cv = CountVectorizer(inputCol='words', outputCol='features', vocabSize=4, minDF=0.1)
model = cv.fit(df_test)
result = model.transform(df_test)
display(result)

In [0]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol='words', outputCol='features')
model = cv.fit(wordsData)
featurizedDF = model.transform(wordsData)
display(featurizedDF)

In [0]:
from pyspark.ml.feature import IDF
from pyspark.sql.functions import col

idf = IDF(inputCol="features", outputCol="idfFeatures")
idfModel = idf.fit(featurizedDF)
rescaledData = idfModel.transform(featurizedDF)
#display(rescaledData)

dataForTraining = rescaledData.select(col('spam').alias('label'), col('idfFeatures').alias('features'))
display(dataForTraining)

In [0]:
trainDF, testDF = dataForTraining.randomSplit([0.8, 0.2])
print(trainDF.count())
print(testDF.count())

In [0]:
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np

lr = LogisticRegression(maxIter = 10)

regParamValues = np.linspace(0.3, 0.01, 4)
print(regParamValues)
elasticNetParamValues = np.linspace(0.3, 0.8, 4)
print(elasticNetParamValues)

paramGr_LR = ParamGridBuilder().addGrid(lr.regParam, regParamValues)\
  .addGrid(lr.elasticNetParam, elasticNetParamValues)\
  .build()

crossValid_LR = CrossValidator(estimator=lr, estimatorParamMaps=paramGr_LR, evaluator=BinaryClassificationEvaluator(), numFolds=5)
cvModel = crossValid_LR.fit(trainDF)
best_model_lr = cvModel.bestModel
best_model_summary = best_model_lr.summary
print(best_model_summary.predictions.columns)